In [1]:
import sqlite3
import pandas as pd
import numpy as np
import pickle
import geopandas
from shapely.geometry import Polygon
import uuid

In [2]:
cnx = sqlite3.connect('../us_wildfire_dataset/FPA_FOD_20170508.sqlite')
df = pd.read_sql_query("SELECT DISCOVERY_DATE, CONT_DATE, LATITUDE, LONGITUDE, STATE, FIRE_NAME, FIRE_SIZE_CLASS, FIRE_SIZE,STAT_CAUSE_DESCR FROM fires", cnx)

In [12]:
def filter_raw_data(df, state='CA', min_class=None):

    # drop states
    df_filt = df[df.STATE == 'CA']
    df_filt = df_filt.drop(['STATE'], axis=1)

    # drop fire classes
    if min_class is not None:
        df_filt.FIRE_SIZE_CLASS = df_filt.FIRE_SIZE_CLASS.apply(ord)
        df_filt = df_filt[df_filt.FIRE_SIZE_CLASS >= ord(min_class)]

    # reformat dates
    df_filt.DISCOVERY_DATE = pd.to_datetime(df['DISCOVERY_DATE'], unit='D', origin='julian')
    df_filt.CONT_DATE = pd.to_datetime(df['CONT_DATE'], unit='D', origin='julian')

    # convert coordinates
    df_filt = geopandas.GeoDataFrame(df_filt, geometry=geopandas.points_from_xy(
        df_filt.LONGITUDE, df_filt.LATITUDE))
    df_filt = df_filt.drop(['LONGITUDE'], axis=1)
    df_filt = df_filt.drop(['LATITUDE'], axis=1)
    df_filt.insert(2, 'COORD', df_filt.pop('geometry'))

    # remove missing values
    df_filt = df_filt.dropna()

    # reformat head
    df_filt.columns = [
        'start_date', 'end_date', 'geometry',
        'name', 'size_class', 'size', 'cause'
    ]

    # sort by start dates
    df_filt = df_filt.sort_values(by='start_date')
    df_filt = df_filt.reset_index()

    return df_filt

In [52]:
def extract_geo_fires(df, area):

    # return fires within polygon
    res = df[df.within(area)]
    return res


def build_geo_grid(df, grid_area, square_size, verbose=False):
    bounds = grid_area.bounds

    # calculate number of grids in lat/long directions
    long_steps = int((bounds[2] - bounds[0]) / square_size)
    lat_steps = int((bounds[3] - bounds[1]) / square_size)

    grid_df = []
    prog, total = 0, long_steps * lat_steps
    for i in range(long_steps):
        for j in range(lat_steps):

            # update progress
            prog += 1
            if verbose and prog % 10 == 0:
                print('Progress: {}/{}'.format(prog, total), flush=True)
            
            # get south-east grid square corner
            c_lon = bounds[0] + i * square_size
            c_lat = bounds[1] + j * square_size

            # create grid square
            grid = Polygon([
                (c_lon, c_lat), 
                (c_lon + square_size, c_lat), 
                (c_lon, c_lat + square_size), 
                (c_lon + square_size, c_lat + square_size)
            ])
            
            fires = extract_geo_fires(df, grid)
            grid_df.append([uuid.uuid4(), grid, fires.index])

    # build grid df
    grid_df = geopandas.GeoDataFrame(grid_df)
    grid_df.columns = ['grid_id', 'grid_square', 'fire_indices']
    return grid_df

In [65]:
# coordinate-square north of San Bernardino/Riverside 
p = Polygon([(-124, 38), (-124, 42), (-120, 38), (-120, 42)])

# filter raw data from dataset and build sexy format
df_filt = filter_raw_data(df, state='CA', min_class='C')

# df_filt is ordered by start_date with the following columns:
#   'start_date', 'end_date', 'geometry', 'name', 'size_class', 'size', 'cause'

# build df with each grid square area (from grid area `p` with grid square size 0.1) and corresponding fires indices (which indexes complete list of fires in df_filt)
grid_df = build_geo_grid(df_filt, p, 0.25, verbose=True)

# grid_df has the following columns:
#    'grid_id', 'grid_square', 'fire_indices'
#    grid_id: a random uuid to identify the grid square for later
#    grid_square: polygon object of the grid square
#    fire_indices: list of indices of the fires from df_filt within the grid_square

Progress: 10/256
Progress: 20/256
Progress: 30/256
Progress: 40/256
Progress: 50/256
Progress: 60/256
Progress: 70/256
Progress: 80/256
Progress: 90/256
Progress: 100/256
Progress: 110/256
Progress: 120/256
Progress: 130/256
Progress: 140/256
Progress: 150/256
Progress: 160/256
Progress: 170/256
Progress: 180/256
Progress: 190/256
Progress: 200/256
Progress: 210/256
Progress: 220/256
Progress: 230/256
Progress: 240/256
Progress: 250/256


In [66]:
def label_grid_square(df, grid_df, start_date='1992-01', end_date='2015-12', verbose=False):
    
    # build date range (in months)
    date_range = [str(d) for d in np.arange(
        start_date, 
        end_date, 
        dtype='datetime64[M]'
    )]    

    # iterate over all grid squares 
    label_df = []
    prog, total = 0, grid_df.shape[0]
    for i in range(grid_df.shape[0]):
        
        # update progress
        prog += 1
        if verbose and prog % 10 == 0:
            print('Progress: {}/{}'.format(prog, total), flush=True)
                
        # get grid square fires
        id = grid_df.loc[i, 'grid_id']
        grid_square = grid_df.loc[i, 'grid_square']
        fire_indices = list(grid_df.loc[i, 'fire_indices'])
        fires = df.loc[fire_indices, :]
        
        # collect all months in fire date range
        months = []
        for _, row in fires.iterrows(): 
            start = row.start_date.date()
            end = (row.end_date + pd.DateOffset(months=1)).date()
            months.extend([str(d) for d in np.arange(start, end, dtype='datetime64[M]')])
        fire_months = list(set(months))
        
        # label fire months
        month_labels = []
        for month in date_range:
            if month in fire_months: month_labels.append(1)
            else: month_labels.append(0)
        
        # add labels
        labels = [id, grid_square] + month_labels
        label_df.append(labels)
        
    # build label df
    label_df = pd.DataFrame(label_df)
    label_df.columns = ['grid_id', 'grid_square'] + date_range
    return label_df

In [81]:
    
# labels whether there was a fire in each grid square 
label_df = label_grid_square(df_filt, grid_df, start_date='2000-01', end_date='2015-12', verbose=False)

# label_df has the following columns:
#    'grid_id', 'grid_square', 'months ...'
#    grid_id: a random uuid to identify the grid square for later
#    grid_square: polygon object of the grid square
#    indices: the rest of the columns are a label for each month (0 if there was no fire in the grid square during them onth and 1 otherwise)

In [86]:
output = []
num_zeros = 0
num_subsequent_1s = 0
total_iters = 0
for index, row in label_df.iterrows():
    # because we are using Landsat 7, let's make sure we constrain dates to be from 2000 onwards (hence the + 96)

    if np.sum(row[2:]) <= 3: continue
    for rowIndex, col in enumerate(row[2 + 96:-1]):
        # if not on fire
        total_iters += 1
        if col == 0:
            num_zeros += 1
            # include implicit conversion to tuple of bounds instead of polygon shape; will make working with earth engine easier
            output.append((row[0], row[1].bounds, label_df.columns[rowIndex + 2 + 96], row[rowIndex + 1 + 2 + 96]))
            if row[rowIndex + 1 + 2 + 96] == 1:
                num_subsequent_1s += 1

In [88]:
print('Total iterations: {}'.format(total_iters))
print('Total data points: {}'.format(num_zeros))
print('Total data points with 1-label: {}'.format(num_subsequent_1s))
print('Total data points with 0-label: {}'.format(num_zeros - num_subsequent_1s))
print('Percentage of 1s: {}%'.format(round(num_subsequent_1s / num_zeros * 100, 3)))

Total iterations: 1504
Total data points: 1380
Total data points with 1-label: 62
Total data points with 0-label: 1318
Percentage of 1s: 0.045%


In [11]:
# save the corresponding data structure to disk
import pickle
with open("../us_wildfire_dataset/labelled_temporal_polygons.pkl", "wb") as f:
    pickle.dump(output, f, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
# load output back from the pickle
with open("../us_wildfire_dataset/labelled_temporal_polygons.pkl", "rb") as f:
    unpickler = pickle.Unpickler(f)
    output = unpickler.load()

In [66]:
import ee
ee.Initialize()

In [89]:
from calendar import monthrange

# change this number 10 to eventually download all of the required images.
for index, item in enumerate(output):
    uuid, coords, date, label = item
    year, month = date.split("-")
    year, month = int(year), int(month)

    geometry = ee.Geometry.Rectangle(list(coords))

    # because we are using LANDSAT7, we need to constrain our dates from 2000 onwards; so far, this has been done above (in cell 96)
    collection = ee.ImageCollection("LANDSAT/LE07/C01/T1").filterDate(ee.Date.fromYMD(year, month, 1), ee.Date.fromYMD(year, month, monthrange(year, month)[1])).filterBounds(geometry)
    trueColor = collection.select(['B3', 'B2', 'B1'])
    col_mean = trueColor.mosaic()

    # images in the drive have been named corresponding to their index in the above output data structure
    task = ee.batch.Export.image.toDrive(
        image = col_mean,             
        region = geometry,
        description = '%d' % (index),
        folder = "world_images",
        scale = 30
    )

    task.start()

In [97]:
task.status()

{'state': 'COMPLETED',
 'description': '886',
 'creation_timestamp_ms': 1589988656129,
 'update_timestamp_ms': 1589988692038,
 'start_timestamp_ms': 1589988661782,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/#folders/1MYfA2k6Hh9xjQQE9nSvcX7Lz-RMACeBc'],
 'id': 'VJU2UMGUZ2VC3XQ7WWS5MVHB',
 'name': 'projects/earthengine-legacy/operations/VJU2UMGUZ2VC3XQ7WWS5MVHB'}